# Getting City Elevation Data
## Steps
1. Determine elevation for each tree via longitude and latitude
2. Save new field, `elevation`, into tree files

In [11]:
import requests

def get_elevation(lat, long):
    query = f"https://api.open-elevation.com/api/v1/lookup?locations={lat},{long}"
    response = requests.get(query).json()
    return response['results'][0]['elevation']

# Example usage:
elevation = get_elevation(40.7142700, -74.0059700)  # Coordinates for New York City
print(f"Elevation: {elevation} meters")

Elevation: 51.0 meters


In [25]:
# Now, let's loop over each file
import os
FILEPATH = "/Users/owen/Development/tmp/bio-data-analysis/data/trees/" # Update with your project path
csv_files = os.listdir(FILEPATH)
csv_files

['Milwaukee_Final_2022-06-18.csv',
 'Anaheim_Final_2022-06-18.csv',
 'Irvine_Final_2022-06-18.csv',
 'RanchoCucamonga_Final_2022-06-18.csv',
 'Arlington_Final_2022-06-18.csv',
 'Rochester_Final_2022-06-18.csv',
 'Miami_Final_2022-06-18.csv',
 'Austin_Final_2022-06-18.csv',
 'Stockton_Final_2022-06-18.csv',
 'Seattle_Final_2022-06-18.csv',
 'LosAngeles_Final_2022-06-18.csv',
 'AuroraCO_Final_2022-06-18.csv',
 'NewYork_Final_2022-06-18.csv',
 'Oakland_Final_2022-06-18.csv',
 'SanFrancisco_Final_2022-06-18.csv',
 'Boston_Final_2022-06-18.csv',
 'Ontario_Final_2022-06-18.csv',
 'GrandRapids_Final_2022-06-18.csv',
 'Baltimore_Final_2022-06-18.csv',
 'Albuquerque_Final_2022-06-18.csv',
 'Greensboro_Final_2022-06-18.csv',
 'OklahomaCity_Final_2022-06-18.csv',
 'HuntingtonBeach_Final_2022-06-18.csv',
 'Fresno_Final_2022-06-18.csv',
 'Orlando_Final_2022-06-18.csv',
 'Nashville_Final_2022-06-18.csv',
 'StLouis_Final_2022-06-18.csv',
 'Tampa_Final_2022-06-18.csv',
 'LasVegas_Final_2022-06-18.csv'

In [26]:
import csv
for file in csv_files:
    print("Going over file: ", file)
    path = os.path.join(FILEPATH, file)
    with open(path, "r") as f:
        reader = csv.reader(f, delimiter=",")
        lines = list(reader)

    # Add the new 'elevation' field to the header
    header = lines[0]
    header.append('elevation')

    # Add the default elevation value to each row
    for row in lines[1:]:
        # Longitude is at index 6
        # Latitude at index 7
        long, lat = row[6], row[7]
        elevation = get_elevation(lat, long)
        row.append(elevation)

    # Write the updated content back to the file
    with open(path, "w", newline="") as f:
        writer = csv.writer(f, delimiter=",")
        writer.writerows(lines)


Going over file:  Milwaukee_Final_2022-06-18.csv


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))